# Problem Statement

Que.4 -

Data Description:
RI : refractive index
Na: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
Mg: Magnesium
AI: Aluminum
Si: Silicon
K:Potassium
Ca: Calcium
Ba: Barium
Fe: Iron
Type: Type of glass: (class attribute)
 1 -- building_windows_float_processed
 2 --building_windows_non_float_processed
 3 --vehicle_windows_float_processed
 4 --vehicle_windows_non_float_processed (none in this database)
 5 --containers
 6 --tableware
 7 --headlamps

Dataset : Glass.csv

Prepare model by using AdaBoost,CatBoost and XGBoost & Calculate Accuracy by confusion matrix, Calculate accuracy score, precision score, recall score, f1 score.

### import the require packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load DataSet

In [2]:
df = pd.read_csv("glass.csv")
df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


### Perform EDA 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


In [4]:
df.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
count,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,1.518365,13.407850,2.684533,1.444907,72.650935,0.497056,8.956963,0.175047,0.057009,2.780374
std,0.003037,0.816604,1.442408,0.499270,0.774546,0.652192,1.423153,0.497219,0.097439,2.103739
min,1.511150,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000,1.000000
25%,1.516522,12.907500,2.115000,1.190000,72.280000,0.122500,8.240000,0.000000,0.000000,1.000000
50%,1.517680,13.300000,3.480000,1.360000,72.790000,0.555000,8.600000,0.000000,0.000000,2.000000
75%,1.519157,13.825000,3.600000,1.630000,73.087500,0.610000,9.172500,0.000000,0.100000,3.000000
max,1.533930,17.380000,4.490000,3.500000,75.410000,6.210000,16.190000,3.150000,0.510000,7.000000


In [5]:
df.isna().sum()

RI      0
Na      0
Mg      0
Al      0
Si      0
K       0
Ca      0
Ba      0
Fe      0
Type    0
dtype: int64

In [6]:
df.corr()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
RI,1.000000,-0.191885,-0.122274,-0.407326,-0.542052,-0.289833,0.810403,-0.000386,0.143010,-0.164237
Na,-0.191885,1.000000,-0.273732,0.156794,-0.069809,-0.266087,-0.275442,0.326603,-0.241346,0.502898
Mg,-0.122274,-0.273732,1.000000,-0.481799,-0.165927,0.005396,-0.443750,-0.492262,0.083060,-0.744993
Al,-0.407326,0.156794,-0.481799,1.000000,-0.005524,0.325958,-0.259592,0.479404,-0.074402,0.598829
Si,-0.542052,-0.069809,-0.165927,-0.005524,1.000000,-0.193331,-0.208732,-0.102151,-0.094201,0.151565
K,-0.289833,-0.266087,0.005396,0.325958,-0.193331,1.000000,-0.317836,-0.042618,-0.007719,-0.010054
Ca,0.810403,-0.275442,-0.443750,-0.259592,-0.208732,-0.317836,1.000000,-0.112841,0.124968,0.000952
Ba,-0.000386,0.326603,-0.492262,0.479404,-0.102151,-0.042618,-0.112841,1.000000,-0.058692,0.575161
Fe,0.143010,-0.241346,0.083060,-0.074402,-0.094201,-0.007719,0.124968,-0.058692,1.000000,-0.188278
Type,-0.164237,0.502898,-0.744993,0.598829,0.151565,-0.010054,0.000952,0.575161,-0.188278,1.000000


In [7]:
df.columns

Index(['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type'], dtype='object')

### split data into x and y 

In [24]:
x = df[['Na', 'Mg', 'Al', 'Ba']]
print(x.head())
y = df['Type']
y.head()

from sklearn.preprocessing import LabelEncoder

y= pd.DataFrame(LabelEncoder().fit_transform(y))
y.head()

      Na    Mg    Al   Ba
0  13.64  4.49  1.10  0.0
1  13.89  3.60  1.36  0.0
2  13.53  3.55  1.54  0.0
3  13.21  3.69  1.29  0.0
4  13.27  3.62  1.24  0.0


,0
0,0
1,0
2,0
3,0
4,0


## Split data into training and testing

In [25]:
from sklearn.model_selection import train_test_split
X_train, x_test, Y_train, y_test = train_test_split(x, y, train_size= 0.7, random_state=153234)

## Built the models

In [26]:
class Models():
    def __init__(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train

    def AdaBoost(self):
        from sklearn.ensemble import AdaBoostClassifier
        model_ada = AdaBoostClassifier().fit(self.X_train, self.Y_train)
        return model_ada

    def Cat_Boost(self):
        from catboost import CatBoostClassifier
        model_cat = CatBoostClassifier().fit(self.X_train, self.Y_train)
        return model_cat

    def Xg_Boost(self):
        from xgboost import XGBClassifier
        model_xg = XGBClassifier().fit(self.X_train, self.Y_train)
        return model_xg

In [30]:
def model_evaluation(model, model_name):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    Accuracy = accuracy_score(y_test, model.predict(x_test))
    Precision = precision_score(y_test, model.predict(x_test), average='micro')
    Recall = recall_score(y_test, model.predict(x_test), average='micro')
    F1_Score = f1_score(y_test, model.predict(x_test), average='micro')
    return model_name, Accuracy, Precision, Recall, F1_Score

In [31]:
m1 = Models(X_train, Y_train)

models = [(m1.AdaBoost(),"AdaBoost"),(m1.Cat_Boost(),"CatBoost"),(m1.Xg_Boost(),"XGBoost")]


/home/sunbeam/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sunbeam/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Learning rate set to 0.071805
0:	learn: 1.7206038	total: 2.09ms	remaining: 2.09s
1:	learn: 1.6538554	total: 3.98ms	remaining: 1.99s
2:	learn: 1.6018208	total: 5.67ms	remaining: 1.88s
3:	learn: 1.5533376	total: 6.61ms	remaining: 1.65s
4:	learn: 1.5016068	total: 8.47ms	remaining: 1.69s
5:	learn: 1.4624944	total: 9.99ms	remaining: 1.66s
6:	learn: 1.4244223	total: 11.7ms	remaining: 1.66s
7:	learn: 1.3870796	total: 13.4ms	remaining: 1.66s
8:	learn: 1.3557462	total: 15ms	remaining: 1.65s
9:	learn: 1.3230876	total: 16.7ms	remaining: 1.65s
10:	learn: 1.2894594	total: 18ms	remaining: 1.62s
11:	learn: 1.2611032	total: 19.6ms	remaining: 1.61s
12:	learn: 1.2347118	total: 21ms	remaining: 1.6s
13:	learn: 1.2097808	total: 22.4ms	remaining: 1.58s
14:	learn: 1.1893939	total: 24ms	remaining: 1.58s
15:	learn: 1.1683166	total: 25.6ms	remaining: 1.57s
16:	learn: 1.1422661	total: 27.1ms	remaining: 1.57s
17:	learn: 1.1258339	total: 28.6ms	remaining: 1.56s
18:	learn: 1.1075024	total: 30.2ms	remaining: 1.56s
1

In [32]:
results = []
for model, model_name in models:
    results.append(model_evaluation(model, model_name))

df1 = pd.DataFrame(results, columns=['Algorithm','Accuracy_Score','Precision_Score','Recall_Score','F1_Score'])
df1 

,Algorithm,Accuracy_Score,Precision_Score,Recall_Score,F1_Score
0,AdaBoost,0.523077,0.523077,0.523077,0.523077
1,CatBoost,0.692308,0.692308,0.692308,0.692308
2,XGBoost,0.738462,0.738462,0.738462,0.738462
